**Notebook preparation and importing libraries**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CoolPropWarning

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
Q_ = Quantity

In [4]:
import hvac.heat_transfer.heat_exchanger.fin_tube.air_condenser.rating as rating

# Rating of a plain fin-tube counterflow air condenser

Define the type of refrigerant:

In [5]:
R134a = Fluid('R134a')

Define the type of condenser: a plain fin-tube, counterflow heat exchanger.

In [6]:
Condenser = rating.PFT_CO_CND

Create an instance of the condenser model, specifying the dimensions of the frontal area, the number of rows, the geometrical characteristics of the heat transfer surfaces, and the thermal conductivity of the fins (by default aluminium fins are assumed):

In [7]:
condenser = Condenser(
    L1=Q_(0.909, 'm'),
    L3=Q_(0.303, 'm'),
    N_r=4,
    S_t=Q_(25.4, 'mm'),         # vertical distance between tubes
    S_l=Q_(22.0, 'mm'),         # horizontal distance between tubes
    D_i=Q_(8.422, 'mm'),        # inner tube diameter
    D_o=Q_(10.2, 'mm'),         # outer tube diameter
    t_f=Q_(0.3302, 'mm'),       # fin thickness
    N_f=1 / Q_(3.175, 'mm'),    # fin density
    k_f=Q_(237, 'W / (m * K)')  # conductivity of fin material
)

On this instance, we set the known operating conditions:
- the mass flow rate of air through the condenser
- the mass flow rate of refrigerant, which was determined when rating the evaporator
- the state of air entering the condenser
- the state of refrigerant entering the condenser

The result of rating the condenser will contain:
- the state of refrigerant leaving the condenser
- the state of air leaving the condenser
- the heat rejection rate (the heat transfer rate from the refrigerant to the air stream)
- the degree of subcooling
- the flow length of air through the desuperheating, condensing, and subcooling part of the condenser

The rating routine (of the subcooling part) is an iterative routine. The criterion to stop the iterative routine is the tolerable deviation between the last and the previous calculated heat exchanger effectiveness, which can be set through parameter `tol_eps`. Should the tolerable deviation not be reached after a maximum number of iterations, which can be set with parameter `i_max`, a `ValueError` exception will be raised and no result is returned.

In [8]:
r = condenser(
    m_dot_air=Q_(2233.861, 'kg / hr'),
    m_dot_rfg=Q_(99.908, 'kg / hr'),
    air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    rfg_in=R134a(T=Q_(85.0, 'degC'), P=Q_(13.179, 'bar'))
)

In [9]:
print(
    f"air out = {r.air_out.Tdb.to('degC'):~P.1f} DB, "
    f"{r.air_out.RH.to('pct'):~P.0f} RH\n"
    f"refrigerant out = {r.rfg_out.T.to('degC'):~P.1f}\n"
    f"degree of subcooling = {r.dT_sc.to('K'):~P.2f}\n"
    f"heat rejection rate = {r.Q_dot.to('kW'):~P.3f}\n"
    f"L2 desuperheating = {r.L2_desuperheating.to('mm'):~P.0f}\n"
    f"L2 condensing = {r.L2_condensing.to('mm'):~P.0f}\n"
    f"L2 subcooling = {r.L2_subcooling.to('mm'):~P.0f}\n"
    f"air-side pressure drop = {r.dP_air.to('Pa'):~P.3f}"
)

air out = 43.9 °C DB, 19 % RH
refrigerant out = 42.6 °C
degree of subcooling = 7.38 K
heat rejection rate = 5.632 kW
L2 desuperheating = 44 mm
L2 condensing = 36 mm
L2 subcooling = 8 mm
air-side pressure drop = 32.178 Pa
